In [ ]:
f = open(path)

In [1]:
import numpy as np
test_ = np.load('/data/data/shhs_new/test111.npz')
val_ = np.load('/data/data/shhs_new/val111.npz')
train_ = np.load('/data/data/shhs_new/train11.npz')


In [2]:
test_name = test_['names']
test_num = test_['nums']
val_name = val_['names']
val_num = val_['nums']
train_name = train_['names']
train_num = train_['nums']

In [3]:
for name in test_name:
    if name in val_name or name in train_name:
        print(name)

In [4]:
for name in train_name:
    if name in val_name or name in test_name:
        print(name)
for name in val_name:
    if name in train_name or name in test_name:
        print(name)

In [5]:
num_sum = 0
for n in val_num:
    num_sum += n
print(num_sum)

91699


In [13]:
91699/(32*32)/20

4.477490234375

In [22]:
import numpy as np
path = '/home/hwx/Sleep/result/idx_Physio2018.npy'
data = np.load(path, allow_pickle=True)

In [27]:
data[0]['val']

array([890, 861, 262, 730, 265, 948, 581, 882, 753, 823, 264, 484, 832,
       266, 525, 593, 443, 828, 942, 847, 604, 731, 819, 915, 834, 229,
       459, 545, 797, 970, 414, 277, 741, 837, 238, 222, 858, 508, 990,
       501, 338, 449, 373, 883, 792, 722, 621, 631, 869, 643])

In [26]:
data[1]['test']

array([199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,
       212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,
       225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
       238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250,
       251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
       264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
       277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289,
       290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302,
       303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315,
       316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328,
       329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341,
       342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354,
       355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367,
       368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 37

In [28]:
x =[0.808, 0.82, 0.799, 0.813, 0.8164]
print(np.mean(x))

0.81128


In [37]:
path_root ="/data/data/SS2"
import mne
import numpy as np
import pyarrow as pa
import os
import glob
import pandas as pd
import gc

In [11]:
ana_spindle = os.path.join(path_root, 'SS2_ana')
spindle_path_E1 = glob.glob(ana_spindle+'/*Spindles_E1*')
spindle_path_E2 = glob.glob(ana_spindle+'/*Spindles_E2*')

In [14]:
spindle_path_E1 = sorted(spindle_path_E1)
spindle_path_E2 = sorted(spindle_path_E2)

In [15]:
s = mne.read_annotations(spindle_path_E1[0])

In [18]:
s.onset

array([  888.569501,   905.999757,   917.97327 , ..., 28184.738132,
       28449.628879, 28623.066266])

In [22]:
epoch = os.path.join(path_root, 'SS2_bio')
name = os.path.split(spindle_path_E1[0])[1].split(' ')[0]
mne.io.read_raw_edf(glob.glob(epoch+f"/{name}*PSG*")[0])

Extracting EDF parameters from /data/data/SS2/SS2_bio/01-02-0001 PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<RawEDF | 01-02-0001 PSG.edf, 27 x 7412736 (28956.0 s), ~31 kB, data not loaded>

In [49]:
E1_anno = []
E2_anno = []
epoch = os.path.join(path_root, 'SS2_bio')
for _, path in enumerate([spindle_path_E1, spindle_path_E2]):
    expert = 'E1'
    if _ == 1:
        expert = 'E2'
    for items in path:

        name = os.path.split(items)[1].split(' ')[0]
        print(f"----------{name}-----------")
        epochs = mne.io.read_raw_edf(glob.glob(epoch+f"/{name}*PSG*")[0])
        anno = mne.read_annotations(items)
        print(f"epochs: {epochs}")
        epochs.load_data()
        epochs.filter(l_freq=0.3, h_freq=35, n_jobs='cuda', method='fir')
        epochs = epochs.resample(100)

        bads = epochs.info['bads']
        badsidx = [epochs[_] for _ in bads]
        badsidx = sorted(badsidx)
        print(f'{epochs.info["bads"]}, idx: {badsidx}')
        epochs.rename_channels({'EEG C3-CLE':'C3', 'EEG C4-CLE':'C4', 'EEG F3-CLE':'F3', 'EEG O1-CLE': 'O1', 'EEG Fpz-CLE': 'Fpz', 'EMG Chin':'EMG', 'EEG Pz-CLE':'Pz','EOG Left Horiz':'EOG'})
        epochs.pick(['C3', 'C4', 'EMG', 'EOG', 'F3', 'Fpz', 'O1', 'Pz'])
        labels = np.zeros(len(epochs))
        choose_idx = {}
        n_epochs = len(epochs)//3000
        for i in range(n_epochs):
            choose_idx[i] = 0
        for times in anno:
            begin_ind = epochs.time_as_index(times=times['onset'])[0]
            end_ind = epochs.time_as_index(times=times['onset']+times['duration'])[0]
            bucket_begin = begin_ind//3000
            butcket_end = end_ind//3000
            print(f"bucket_begin: {bucket_begin}, butcket_end: {butcket_end}")
            if butcket_end==bucket_begin:
                if butcket_end in choose_idx:
                    choose_idx[butcket_end] = 1
                    print(f"saving bucket_begin:{bucket_begin}")
            else:
                end_begin = butcket_end*3000
                if (end_begin-begin_ind) //(butcket_end-bucket_begin) > 0.25:
                    if bucket_begin in choose_idx:
                        choose_idx[bucket_begin] = 1
                        print(f"saving bucket_begin:{bucket_begin}, end_begin: {end_begin}, begin_ind:{begin_ind}, butcket_end:{butcket_end}")
                if (butcket_end-end_begin) //(butcket_end-bucket_begin) > 0.25:
                    if butcket_end in choose_idx:
                        choose_idx[butcket_end] = 1
            labels[begin_ind:end_ind] = 1
        print(type(labels))
        epochs = epochs[:, :n_epochs*3000][0]
        print(epochs.shape)
        labels = labels[:n_epochs*3000]
        epochs = np.split(epochs, n_epochs, axis=1)
        labels = np.split(labels, n_epochs, axis=0)
        print(f"epochs.shape: {len(epochs)}")
        print(f'labels: {len(labels)}')
        cnt=0
        filename='/data/data/MASS/SS2' + f"/{expert}"
        for k, v in choose_idx.items():
            if v==1:
                save_epochs = epochs[k]
                save_labels = labels[k]
                dataframe = pd.DataFrame(
                        {'x': [save_epochs.tolist()], 'Spindles': [save_labels.tolist()], 'bads': [bads]}
                    )
                table = pa.Table.from_pandas(dataframe)
                os.makedirs(f"{filename}/{name}", exist_ok=True)
                with pa.OSFile(
                            f"{filename}/{name}/{str(cnt).zfill(5)}.arrow", "wb"
                    ) as sink:
                        with pa.RecordBatchFileWriter(sink, table.schema) as writer:
                            writer.write_table(table)
                cnt += 1
                del dataframe
                del table
                gc.collect()

print('End')


----------01-02-0001-----------
Extracting EDF parameters from /data/data/SS2/SS2_bio/01-02-0001 PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
epochs: <RawEDF | 01-02-0001 PSG.edf, 27 x 7412736 (28956.0 s), ~31 kB, data not loaded>
Reading 0 ... 7412735  =      0.000 ... 28955.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2817 samples (11.004 sec)

Using CUDA for FFT FIR filtering
[], idx: []
bucket_begin: 29, butcket_end: 29
saving bucket_beg

In [58]:
E1_sub = '/data/data/MASS/SS2/E2/*'
names = []
nums = []
for sub in glob.glob(E1_sub):
    names.append(sub)
for name in names:
    print(f'------{name}-------')
    tmp = 0
    for item in os.listdir(name):
        if os.path.isfile(os.path.join(str(name), str(item))):
            tmp += 1
    print(f'num: {tmp}')
    nums.append(tmp)
nums = np.array(nums)

------/data/data/MASS/SS2/E2/01-02-0001-------
num: 497
------/data/data/MASS/SS2/E2/01-02-0002-------
num: 572
------/data/data/MASS/SS2/E2/01-02-0003-------
num: 263
------/data/data/MASS/SS2/E2/01-02-0005-------
num: 383
------/data/data/MASS/SS2/E2/01-02-0006-------
num: 370
------/data/data/MASS/SS2/E2/01-02-0007-------
num: 466
------/data/data/MASS/SS2/E2/01-02-0009-------
num: 442
------/data/data/MASS/SS2/E2/01-02-0010-------
num: 482
------/data/data/MASS/SS2/E2/01-02-0011-------
num: 416
------/data/data/MASS/SS2/E2/01-02-0012-------
num: 416
------/data/data/MASS/SS2/E2/01-02-0013-------
num: 509
------/data/data/MASS/SS2/E2/01-02-0014-------
num: 474
------/data/data/MASS/SS2/E2/01-02-0017-------
num: 463
------/data/data/MASS/SS2/E2/01-02-0018-------
num: 481
------/data/data/MASS/SS2/E2/01-02-0019-------
num: 455


In [59]:
n = len(names)
idx = np.arange(n)
np.random.shuffle(idx)
names = np.array(names)
nums = np.array(nums)
k_split = n//5
res = {}
path = '/data/data/MASS/SS2'
for i in range(5):
    st = i*k_split
    ed = (i+1)*k_split
    idx_split = idx[st:ed]
    idx_train = np.setdiff1d(idx, idx_split)
    res[f'train_{i}'] = {}
    res[f'train_{i}']['names'] = names[idx_train[1:]]
    res[f'train_{i}']['nums'] = nums[idx_train[1:]]
    res[f'val_{i}'] = {}
    res[f'val_{i}']['names'] = names[idx_train[:1]]
    res[f'val_{i}']['nums'] = nums[idx_train[:1]]
    res[f'test_{i}'] = {}
    res[f'test_{i}']['names'] = names[idx_split]
    res[f'test_{i}']['nums'] = nums[idx_split]
    print(len(res[f'test_{i}']['nums']), len(res[f'test_{i}']['names']), len(res[f'val_{i}']['nums']), len(res[f'train_{i}']['nums']))
np.save(os.path.join(path, f'all_split_E2'), arr= res,allow_pickle=True)

3 3 1 11
3 3 1 11
3 3 1 11
3 3 1 11
3 3 1 11


In [61]:
save_epochs*1e6

array([[12.59348165, 12.24576455, 10.75828768, ..., -5.30419352,
        -7.77917926, -9.80996613],
       [14.74600827, 14.20694958, 11.28172268, ..., -1.28705963,
        -5.23644284, -6.38107045],
       [-0.18425495,  1.05206268,  0.6697732 , ...,  0.064568  ,
        -0.63914808,  0.57524177],
       ...,
       [-3.29206532, -0.33519855,  0.43334084, ..., 10.64025459,
         6.28685872,  4.29418032],
       [13.17502022, 11.5569727 ,  8.30966907, ..., -0.09902951,
         0.96922375,  5.56036744],
       [18.12666523, 19.52697618, 15.8730311 , ..., 26.00480445,
        24.12783658, 23.79564619]])